In [19]:
import matplotlib    
from helpers import *
import imageio  
import matplotlib.pyplot as plt  
import os     
import numpy  
import pandas 
from plts import *   

from IPython.display import IFrame 
import abc   
import random  
from operator import itemgetter 
from functools import partial 
import matplotlib.pyplot as plt
import os   

# credit   

""" work original published by on https://github.com/DiegoVicen/som-tsp' """

" work original published by on https://github.com/DiegoVicen/som-tsp' "

In [2]:
def plot_map(directory, cities, neurons, iteration):
    """
    Generates the required map of cities and neurons at a given moment and
    stores the result in a png image. The map contains all the cities
    represented as red dots and all the neurons as green, crossed by a line    
    dots. The method plots the iteration in which the snapshot was taken.
    :param cities: the cities to be plotted, passed as a list of (x, y)
    coordinates
    :param neurons: the cities to be plotted, passed as a list of (x, y)
    coordinates
    :param iteration: the iterations when the snapshot is taken
    :return: returns nothing
    """
    plt.scatter(*zip(*cities), color='red', s=3)
    plt.scatter(*zip(*neurons), color='green', s=2)

    plt.plot(*zip(*(neurons+[neurons[0]])), color='darkgreen')

    # Invert x axis to match representation at
    # http://www.math.uwaterloo.ca/tsp/world/countries.html
    plt.gca().invert_xaxis()
    plt.gca().set_aspect('equal', adjustable='datalim')

    plt.title('Iteration #{:06d}'.format(iteration))
    plt.axis('off') 
    plt.savefig('{}/{}.png'.format(directory, iteration))
    plt.clf() 

In [3]:
COUNTRY = 'western_sahara' # could be either of the two
SET_ID = 'w' 

# starting points for the decay functions have been taken 
# from a prior example 
DEFAULTS_EXP = {
            'western_sahara': ('western_sahara', 8, 500, 50, 50, gaussian,
                        ExponentialDecay(0.7, 0.999),
                        ExponentialDecay (29 * 8/10, 0.995)),
    
            'qatar': ('qatar', 8, 5000, 500, 500, gaussian,
                        ExponentialDecay(0.9, 0.9999),
                        ExponentialDecay(194*8/10, 0.997))
            }

In [4]:
def plot_map(directory, cities, neurons, iteration):
    """
    Generates the required map of cities and neurons at a given moment and
    stores the result in a png image. The map contains all the cities
    represented as red dots and all the neurons as green, crossed by a line
    dots. The method plots the iteration in which the snapshot was taken.
    :param cities: the cities to be plotted, passed as a list of (x, y)
    coordinates
    :param neurons: the cities to be plotted, passed as a list of (x, y)
    coordinates
    :param iteration: the iterations when the snapshot is taken
    :return: returns nothing
    """
    plt.scatter(*zip(*cities), color='red', s=3)
    plt.scatter(*zip(*neurons), color='green', s=2)

    plt.plot(*zip(*(neurons+[neurons[0]])), color='darkgreen')

    # Invert x axis to match representation at
    # http://www.math.uwaterloo.ca/tsp/world/countries.html
    plt.gca().invert_xaxis()
    plt.gca().set_aspect('equal', adjustable='datalim')

    plt.title('Iteration #{:06d}'.format(iteration))
    plt.axis('off') 
    plt.savefig('{}/{}.png'.format(directory, iteration))
    plt.clf()  

In [5]:
plt.figure() 

# x, y co-ordinate for every city in the country 
def read_data(filename):
    """
    Reads and parses data from a txt file with a map data. The format that the
    function expects is the one followed in the uwaterloo TSP web archive
    (math.uwaterloo.ca/tsp/world/countries.html), ignoring the first
    description lines that have to be manually removed. The method searches for
    the filename in the assets folder.
    :param filename: the path to the file to be parsed
    :return: the cities as a list of (x, y) coordinates
    """
    cities = [] 

    path = 'assets/{}.txt'.format(filename)
    with open(path, 'r') as f:
        for line in f:  
            city = list(map(float, line.split()[1:]))
            cities.append((city[1], city[0]))
    return cities    

def get_input():
    """
    Gets the input from the user line or launches the default values
    :return data_set: list of cities as (x,y) coordinates
    :return n_neurons: number of neurons per city in the data_set
    :return iterations: number of iterations to be executed
    :return learning_rate: learning rate to be used
    :return radius: radius of neurons to be used
    """
    data_sets = {'w': 'western_sahara', 'q': 'qatar'} 
    data_set = data_sets[SET_ID] 

    if not os.path.isfile('assets/{}.txt'.format(data_set)):
        exit("Did not find this data set file!") 

    return DEFAULTS_EXP[data_set] 

<Figure size 432x288 with 0 Axes>

In [6]:
def main(IMG_DIR): 
    data_set, n_neurons, iterations, k, plot_k, neighborhood, learning_rate\
        , radius = get_input()  
    cities = read_data(data_set)  
    scaling, cities = normalize(cities)
    neuron_count = len(cities) * n_neurons 
    neurons = init_neurons(neuron_count)

    # TODO maybe distance param? 
    som(neurons, cities, iterations, k, plot_k, neighborhood, learning_rate, radius,
        scaling, IMG_DIR)  


def init_neurons(count): 
    """
        Initialize the weights of the neurons 
        :param count: number of neurons to initialize
        :return: list of neurons as weight vectors (x, y)
    """ 
    return [[random.uniform(0.0, 1.0),
             random.uniform(0.0, 1.0)] for i in range(count)]


def som(neurons, cities, iterations, k, plot_k, neighborhood, learning_rate,
        radius, scaling, IMG_DIR):
    """
    Main SOM loop to be executed.
    :param neurons: list of neurons as weight vectors (x,y)
    :param cities: list of cities as (x,y) coordinates
    :param iterations: number of iterations to perform
    :param k: lapse of iterations between printing execution data
    :param plot_k: lapse of iterations between saving the map images
    :param neighborhood: type of neighborhood to use in the execution
    :param learning_rate: learning rate to be used
    :param radius: radius of neurons to be used
    :param scaling: scale used when normalizing the data
    :return: returns nothing
    """
    for i in range(0, iterations+1):
        if i % k == 0:
            print('#', i, '\t\tTSP-distance: ',
                  calculate_tsp(cities, neurons)*scaling)
        if i % plot_k == 0: 
            print('plotting to file {}'.format(IMG_DIR))
            plot_map(IMG_DIR, cities, neurons, i)
        if i == iterations:
            break 
        som_iteration(neurons, cities, neighborhood, learning_rate, radius)


def som_iteration(neurons, cities, neighborhood, learning_rate, radius):
    """ 
     Performs a single SOM iteration with a given set of parameters
    :param neurons: list of neurons as weight vectors (x,y)
    :param cities: list of cities as (x,y) coordinates
    :param neighborhood: type of neighborhood to use in the execution
    :param learning_rate: learning rate to be used
    :param radius: radius of neurons to be used
    :return: returns nothing
    """
    # Pick a random city   
    city = cities[random.randint(0, len(cities)-1)]  
    
    # Choose the winner neuron
    winner_index, winner = compute_winner(city, neurons, euclidean_distance_2d)
    distance = partial(euclidean_distance_1d_circular, len(neurons)) 
    
    # Update the weights of the neuron and its neighbourhood
    for neuron_index, neuron in enumerate(neurons):   
        d = distance(neuron_index, winner_index) 
        nf = neighborhood(d, radius.value)     
        neuron[0] += learning_rate.value * nf * (city[0] - neuron[0])
        neuron[1] += learning_rate.value * nf * (city[1] - neuron[1])
    
    learning_rate.decay()    
    radius.decay()      

def compute_winner(city, neurons, distance):
    """
    Computes the closest neuron to a given city with a given distance
    :param city: coordinates (x,y) of the city to be associated to the winner
    :param neurons: list of neurons as weight vectors (x,y)
    :param distance: type of distance to be used for the computations
    :return winner_index: index of the chosen neuron in the neuron list
    :return winner: closest neuron to city
    """
    return min([(i, distance(city, neuron)) for i,
                neuron in enumerate(neurons)], key=itemgetter(1))


def normalize(cities):
    """
    Normalize list of city coordinates
    :param cities: list of tuples, containing x and y coordinate
    :return: normalized list of city coordinates
    """
    max_x = max(cities,key=itemgetter(0))[0]
    max_y = max(cities,key=itemgetter(1))[1]
    m = max(max_x, max_y)  
    return m, [(x/m, y/m) for (x, y) in cities]


def calculate_tsp(cities, neurons):
    """
    Computes the Travelling Salesman distance to travel all the cities using a
    list of neurons
    :param cities: list of tuples, containing x and y coordinate
    :param neurons: list of neurons as weight vectors (x,y)
    :return: the distance to travel through cities using neurons
    """ 
    city_neurons = {}
    
    for city_idx, city in enumerate(cities):
        # find nearest neuron
        idx, _ = compute_winner(city, neurons, euclidean_distance_2d)
        
        if idx not in city_neurons:
            city_neurons[idx] = [city]
        else: 
            city_neurons[idx].append(city)

    # order cities according to neuron order
    tsp_order = []
    for neuron_idx in range(len(neurons)):
        if neuron_idx in city_neurons:
            tsp_order += city_neurons[neuron_idx]

    # calculate tsp distance for tsp_order
    tsp_distance = euclidean_distance_2d(tsp_order[0], tsp_order[-1])
    for idx in range(len(tsp_order)-1):
        tsp_distance += euclidean_distance_2d(tsp_order[idx],
                                              tsp_order[idx + 1])

    return tsp_distance   

In [20]:
DIR = "display_dir_5" 
gif_name = "display_gif_5" 
! mkdir $DIR 

In [21]:
main(DIR)    

# 0 		TSP-distance:  61478.472625640614
plotting to file display_dir_5
# 50 		TSP-distance:  57782.603402871784
plotting to file display_dir_5
# 100 		TSP-distance:  51922.048128175054
plotting to file display_dir_5
# 150 		TSP-distance:  56601.23099245254
plotting to file display_dir_5
# 200 		TSP-distance:  58409.396111613685
plotting to file display_dir_5
# 250 		TSP-distance:  58675.130037843126
plotting to file display_dir_5
# 300 		TSP-distance:  59531.64066374555
plotting to file display_dir_5
# 350 		TSP-distance:  59026.837087309024
plotting to file display_dir_5
# 400 		TSP-distance:  59026.837087309024
plotting to file display_dir_5
# 450 		TSP-distance:  59026.837087309024
plotting to file display_dir_5
# 500 		TSP-distance:  59026.837087309024
plotting to file display_dir_5


<Figure size 432x288 with 0 Axes>

In [22]:
import imageio
from PIL import Image 

def display_gif(path, gif_name):
    dirFiles = [x for x in os.listdir(path) if '.png' in x]
    lsorted = sorted(dirFiles, key=lambda x: int(os.path.splitext(x)[0]))
    lsorted =[f'/Users/iqrashahzad/Desktop/SOM_TSP/src/{path}/'+ x for x in lsorted] 
    frames = []
    for im in lsorted:  
        frames.append(imageio.imread(im))
        
    imageio.mimsave(f'{gif_name}.gif', frames, fps=1)  
    print('Gif saved ')

In [23]:
display_gif(DIR, gif_name)  

Gif saved 


In [26]:
# print

In [17]:
IFrame(f'{gif_name}.gif', width=700, height=350)       